In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp template.formatter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# export
from pathlib import Path
from typing import Dict, Union
from fastcore.script import call_parse, Param
import zipfile
import tempfile
import urllib
from string import Template
import re

# Template Formatter

This formatter makes getting started with plugins easier, by generating a plugin template with an easy to use CLI. setup files, docker files and the plugin structure are generated for you.

Example usage:
```
indexer_from_template --name <gitlab-username> --plugin_name "My Indexer Plugin"
```

In [ ]:
# export
INDEXER_TEMPLATE_URL = "https://gitlab.memri.io/plugins/indexer-template/-/archive/master/indexer-template-master.zip"

In [ ]:
# export
def str_to_identifier(s, lower=True):
    result = re.sub("\W|^(?=\d)", "_", s)
    if lower:
        result = result.lower()
    return result

def download_indexer_template(
    url=INDEXER_TEMPLATE_URL, base_path="indexer-template-master/template/"
):
    zip_path, _ = urllib.request.urlretrieve(url)
    with zipfile.ZipFile(zip_path, "r") as f:
        result = {name: f.read(name) for name in f.namelist()}
        result = {
            k.replace(base_path, ""): v.decode("utf-8") for k, v in result.items() if v
        }
    return result

In [ ]:
str_to_identifier("My Plugin")

'my_plugin'

In [ ]:
# export
class TemplateFormatter:
    def __init__(
        self,
        template_dict: Dict[str, str],
        replace_dict: Dict[str, str],
        tgt_path: Union[str, Path],
        verbose: bool = False,
    ):
        self.template_dict = template_dict
        self.tgt_path = Path(tgt_path)
        self.replace_dict = replace_dict
        self.verbose = verbose

    def format_content(self, content):
        return Template(content).substitute(self.replace_dict)

    def format_path(self, path):
        new_path = Template(path).substitute(self.replace_dict)
        return self.tgt_path / new_path

    def format_file(self, filename, content):
        new_path = self.format_path(filename)
        new_content = self.format_content(content)
        new_path.parent.mkdir(exist_ok=True, parents=True)
        if self.verbose:
            print(f"Formatting {filename} -> {new_path}")
        with open(new_path, "w", encoding="utf-8") as f:
            f.write(new_content)

    def format(self):
        for filename, content in self.template_dict.items():
            self.format_file(filename, content)

In [ ]:
# export
@call_parse
def indexer_from_template(
    user: Param("Your Gitlab username", str) = None,
    repo_name: Param("The name of your Gitlab plugin repository", str) = None,
    package_name: Param("Name of your plugin python package", str) = None,
    plugin_name: Param("Display name of your plugin", str) = None,
    model_name: Param("Name of the model used in the indexer", str) = None,
    target_dir: Param("Directory to output the formatted template", str) = ".",
):
    if user is None:
        print("Define your gitlab user name with `--user <username>`")
        return

    if plugin_name is None:
        print("Define your gitlab user name with `--plugin_name <name>`")
        return

    if package_name is None:
        package_name = str_to_identifier(plugin_name, lower=True)

    if repo_name is None:
        repo_name = package_name

    if model_name is None:
        model_name = package_name + "_model"

    template = download_indexer_template()
    tgt_path = Path(target_dir)
    replace_dict = {
        "user": user,
        "repo_name": repo_name,
        "package_name": package_name,
        "plugin_name": plugin_name,
        "model_name": model_name,
    }

    formatter = TemplateFormatter(template, replace_dict, tgt_path)
    formatter.format()

In [ ]:
# skip
template = download_indexer_template()
result_path = Path("./result/")
replace_dict = {
    "user": "eelcovdw",
    "repo_name": "sentiment-plugin",
    "package_name": "sentiment_plugin",
    "plugin_name": "Sentiment Plugin",
    "model_name": "cardiffnlp/twitter-xlm-roberta-base-sentiment",
}

formatter = TemplateFormatter(template, replace_dict, result_path, True)
formatter.format()

Formatting $package_name/model.py -> result/sentiment_plugin/model.py
Formatting Dockerfile -> result/Dockerfile
Formatting README.md -> result/README.md
Formatting plugin_config.json -> result/plugin_config.json
Formatting setup.cfg -> result/setup.cfg
Formatting setup.py -> result/setup.py
Formatting tools/preload.py -> result/tools/preload.py


In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.photo.ipynb.
Converted index.ipynb.
Converted itembase.ipynb.
Converted plugin.authenticators.credentials.ipynb.
Converted plugin.authenticators.oauth.ipynb.
Converted plugin.listeners.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.states.ipynb.
Converted plugins.authenticators.password.ipynb.
Converted pod.api.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
Converted template.formatter.ipynb.
Converted test_schema.ipynb.
Converted test_utils.ipynb.
